# Passing input arguments to your QiskitFunction

In this document, we will learn how to pass arguments to our function.

Let's create another file with our function [./source_files/function_with_arguments.py](./source_files/function_with_arguments.py). 

Instead of having the circuit defined inside the function (like we did in the first example), we will pass it as an argument. To make this case more realistic, we will also pass a backend argument for the transpilation and the Sampler. This use case is for transpiling a circuit in the cloud. We will also save the results, so we can access them later by calling [save_result](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.core.save_result.html#qiskit_serverless.core.save_result).

Here is the function:

```python
"""function with arguments for jupyter notebook."""
from qiskit import QuantumCircuit
from qiskit.providers.exceptions import QiskitBackendNotFoundError
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_serverless import get_arguments, save_result
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime.fake_provider import FakeProviderForBackendV2
from qiskit_ibm_runtime import SamplerV2 as Sampler


# get all arguments passed to this function
arguments = get_arguments()

# get specific argument that we are interested in
circuit = arguments.get("circuit")
backend_name = arguments.get("backend_name")
service = arguments.get("service")

# verifying arguments types
if not isinstance(circuit, QuantumCircuit):
    raise ValueError("circuit must be QuantumCircuit.")
if not isinstance(backend_name, str):
    raise ValueError("backend_name must be str.")

if "fake" in backend_name.lower():
    service = FakeProviderForBackendV2()
if isinstance(service, (FakeProviderForBackendV2, QiskitRuntimeService)):
    try:
        backend = service.backend(backend_name)
    except QiskitBackendNotFoundError as e:
        raise ValueError(f"Error retrieving backend {backend_name}: {e}") from e
else:
    raise ValueError(f"A service of type `QiskitRuntimeService` is required for using the backend named {backend_name}.")

# matching our run to the backend argument
sampler = Sampler(backend)
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
isa_circuit = pm.run(circuit)

# running the circuit
quasi_dists = sampler.run([isa_circuit]).result()[0].data.meas.get_counts()

# saving results of the execution
save_result({
    "quasi_dists": quasi_dists
})

```

As you can see, the circuit construction is not inside the function anymore. Instead, we parse the arguments by calling the [get_arguments](https://qiskit.github.io/qiskit-serverless/stubs/qiskit_serverless.serializers.get_arguments.html#qiskit_serverless.serializers.get_arguments) function.

First, we will create circuit that we want to pass as an argument to the function.

In [1]:
from qiskit import QuantumCircuit

circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

┌───┐      ░ ┌─┐   
   q_0: ┤ H ├──■───░─┤M├───
        └───┘┌─┴─┐ ░ └╥┘┌─┐
   q_1: ─────┤ X ├─░──╫─┤M├
             └───┘ ░  ║ └╥┘
meas: 2/══════════════╩══╩═
                      0  1

Our Function can run on either simulator or a real hardware. For real hardware usage, a service object is needed to connect and send the data to the IQP. Here is how we can do it

In [2]:
from qiskit_ibm_runtime import QiskitRuntimeService
 
service = QiskitRuntimeService(
        # channel="ibm_quantum_platform",
        # token="API_KEY",
        # instance="CRN"
)
backend_name = service.least_busy(simulator=False, operational=True)

management.get:WARNING:2025-10-30 12:34:31,507: Loading default saved account


We decide to use a simulator as this is an example. Because of that we will write oer our backend name and service so our function will run over a simulator.In this ecample, we chose `FakeVigoV2` to be our backend:

In [3]:
from qiskit_ibm_runtime.fake_provider import FakeVigoV2

backend_name = FakeVigoV2().name

Now let's create and configure our client

> &#x26A0; This provider is set up with default credentials to a test cluster intended to run on your machine. For information on setting up infrastructure on your local machine, check out the guide on [local infrastructure setup](https://qiskit.github.io/qiskit-serverless/deployment/local.html).

In [4]:
from qiskit_serverless import ServerlessClient
import os

client = ServerlessClient(
    token=os.environ.get("GATEWAY_TOKEN", "awesome_token"),
    instance=os.environ.get("GATEWAY_INSTANCE", "an_awesome_crn"),
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
    # If you are using the kubernetes approach the URL must be http://localhost
)
client

<gateway-client>

In [5]:
from qiskit_serverless import QiskitFunction

function = QiskitFunction(
    title="function-with-arguments",
    entrypoint="function_with_arguments.py",
    working_dir="./source_files/",
)

client.upload(function)

QiskitFunction(function-with-arguments)

Run the function using the `run` method and by passing arguments.

In [6]:
my_function = client.get("function-with-arguments")
my_function

QiskitFunction(function-with-arguments)

In [7]:
job = my_function.run(circuit=circuit, backend_name=backend_name, service=service)
job

<Job | 8abf9e9f-42ff-4b5b-a395-71aadb009bd8>

Retrieve the results from the client

In [8]:
job.result()

{'quasi_dists': {'11': 504, '00': 520}}

In [9]:
print(job.logs())

2025-10-30 09:42:40,698	INFO job_manager.py:531 -- Runtime env is setting up.

